In [1]:
from rusty_tags import *
from rusty_tags.datastar import Signals
from rusty_tags.utils import create_template, show, page_template

hdrs = (
    Script(src="https://cdn.jsdelivr.net/gh/starfederation/datastar@main/bundles/datastar.js", type="module"),
)
htmlkws = dict(cls="bg-background text-foreground font-sans antialiased")
bodykws = dict(cls="h-screen p-16 bg-white text-foreground font-sans antialiased")
page = create_template(hdrs=hdrs, htmlkw=htmlkws, bodykw=bodykws)
template = page_template(hdrs=hdrs, htmlkw=htmlkws, bodykw=bodykws)

@page(title="D* Demo")
def index():
    return Div(
        H1(text="$message", cls="text-4xl font-bold"),
        signals=Signals(message="Hello, from datastar signals!"),
    )

indexPage = index()
print(indexPage)
show(indexPage)

<!doctype html><html class="bg-background text-foreground font-sans antialiased"><head><title>D* Demo</title><script type="module" src="https://cdn.jsdelivr.net/gh/starfederation/datastar@main/bundles/datastar.js"></script><script src="https://cdn.jsdelivr.net/gh/starfederation/datastar@main/bundles/datastar.js" type="module"></script><script src="https://unpkg.com/lucide@latest"></script></head><body class="h-screen p-16 bg-white text-foreground font-sans antialiased"><div data-signals="{'message':'Hello, from datastar signals!'}"><h1 class="text-4xl font-bold" data-text="$message"></h1></div><script>lucide.createIcons();</script></body></html>


In [2]:
from pydantic import BaseModel, Field
from typing import Optional

class User(BaseModel):
    id: int = Field(description="The unique identifier for the user")
    name: str = Field(description="The name of the user")
    email: Optional[str] = Field(description="The email address of the user")
    age: Optional[int] = 100

for field, field_info in User.model_fields.items():
    print(f"{field.capitalize()}: {field_info.description} - Default: {field_info.default}")



Id: The unique identifier for the user - Default: PydanticUndefined
Name: The name of the user - Default: PydanticUndefined
Email: The email address of the user - Default: PydanticUndefined
Age: None - Default: 100


In [3]:
def display(component):
    return show(template(component))


In [4]:
def render_view(instance: BaseModel):
    components = []
    for field_name, field_info in instance.__class__.model_fields.items():
        value = getattr(instance, field_name)
        components.append(
            Div(
                Label(f"{field_info.title or field_name.capitalize()} :", cls="uk-label text-md"),
                Span(str(value), title=field_info.description or "", cls="text-md"),
                cls="border rounded-md mb-2 p-1"
            )
        )
    return Div(*components)

user = User(id=1, name="John Doe", email="john.doe@example.com")
display(render_view(user))

In [5]:
from rusty_tags.utils import AttrDict
from typing import Any

class Signals(AttrDict):
    """
    A dictionary of signals with reactive capabilities.
    """
    def __new__(cls, *args, **kwargs) -> None:
        instance = super().__new__(cls)
        instance.__init__(*args, **kwargs)
        instance._post_init(*args, **kwargs)
        return instance

    def _post_init(self, *args, **kwargs):
        if "namespace" in kwargs:
            setattr(self, "_namespace", kwargs.pop("namespace"))
        
        super().__init__(**kwargs)

        for k, v in kwargs.items():
            setattr(self, f"_{k}", f"{self._namespace}.{k}" if self._namespace else k)
            setattr(self, f"_S{k}", f"${self._namespace}.{k}" if self._namespace else f"${k}")
        
    def __str__(self):
        return f"{{{self._namespace}: {super().__str__()}}}" if self._namespace else super().__str__()
user = Signals(name="John Doe 2", age=35, namespace="User")

print(f"user._Sname = {user._Sname}")
print(f"user._name = {user._name}")
print(f"user.name = {user.name}")
print(f"user = {user}")


user._Sname = $User.name
user._name = User.name
user.name = John Doe 2
user = {User: {'name': 'John Doe 2', 'age': 35, 'namespace': 'User'}}


In [6]:
user = Signals(name="John Doe 2", age=35)

print(f"user._Sname = {user._Sname}")
print(f"user._name = {user._name}")
print(f"user.name = {user.name}")
print(f"user = {user}")


user._Sname = $name
user._name = name
user.name = John Doe 2
user = {'name': 'John Doe 2', 'age': 35}


In [7]:
usrDict = {'name': 'John Doe 2', 'age': 35}

user = Signals(**usrDict)

print(f"user._Sname = {user._Sname}")
print(f"user._name = {user._name}")
print(f"user.name = {user.name}")


user._Sname = $name
user._name = name
user.name = John Doe 2


In [8]:
usrDict2 = {"user": AttrDict({'name': 'John Doe 2', 'age': 35})}
user2 = Signals(usrDict2)

user2.user
user2.user.name

'John Doe 2'

## Styling

In [9]:
TextT = AttrDict()
'Text Styles from https://franken-ui.dev/docs/text'
    
TextT.paragraph = "uk-paragraph"
# Text Style
TextT.lead, TextT.meta, TextT.gray, TextT.italic= "lead", "meta", 'text-gray-500 dark:text-gray-200', 'italic'
# Text Size
TextT.xs, TextT.sm, TextT.lg, TextT.xl = 'text-xs', 'text-sm', 'text-lg', 'text-xl'
# Text Weight
TextT.light, TextT.normal, TextT.medium, TextT.bold, TextT.extrabold = 'font-light','font-normal','font-medium','font-bold','font-extrabold'
# Text Color
TextT.muted,TextT.primary,TextT.secondary = 'text-gray-500 dark:text-gray-200', 'text-primary', 'text-secondary'


h-24 w-24
h-24 w-24
